In [224]:
import pandas as pd
import requests
import os
import sys

#URLlib
import urllib
import urllib.request

#PDFminer
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
from io import StringIO
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal

In [225]:
# Frases a buscar
phrases_to_seek = {
    "desaceleración economía mundial": 4,
    "perspectivas crecimiento": 4,
    "inflaciones general subyacente principales economías avanzadas": 5,
    "reserva federal": 5,
    "comunicación destacó": 6,
    "mercados financieros internacionales": 6,
    "volatilidad": 4,
    "posturas acomodaticias principales bancos centrales": 4,
    "riesgos economía global": 3,
    "balance riesgos actividad económica mundial": 3,
    "mercados financieros economías emergentes": 4,
    "cotización peso": 4,
    "tasas interés valores gubernamentales": 6,
    "política monetaria prudente firme": 2,
    "consoliden sosteniblemente las finanzas públicas": 3,
    "perspectivas crediticias soberana pemex": 3,
    "actividad económica": 4,
    "holgura economía": 4,
    "balance riesgos crecimiento": 5,
    "inflación general": 5,
    "inflación subyacente": 5,
    "subyacente": 5,
    "persistencia mostrada inflación subyacente": 0,
    "aumentos salariales": 4,
    "finanzas públicas": 4,
    "cotización moneda": 4,
    "mantiene incertidumbre cuanto riesgos pudieran desviar inflación respecto trayectoria esperada": 0,
    "balance de riesgos para la inflación": 3,
    "incertidumbre": 4,
    "condiciones holgura economía": 4,
    "decidió mayoría": 4,
    "decidió unanimidad": 4,
    "decidió manera unánime": 5,
    "un miembro votó": 5,
    "dos miembros votaron": 5,
    "tomará las acciones": 8,
    "convergencia ordenada y sostenida de la inflación general": 6,
    "postura monetaria prudente": 4,
    "ajustará de manera oportuna y firme": 8,
    "fortalecer el anclaje": 5,
    "presiones costos economía": 4,
    "tipo cambio": 4
}

In [226]:
path = "statement.pdf"

In [227]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,
                                  password=password,
                                  caching=caching,
                                  check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [241]:
pdf_text = convert_pdf_to_txt(path).replace('\n', '')

In [242]:
words_to_replace = ["\x0c", ".", ",", " i)", "ii", "iii", "iv)", "v)", "vi)", ")", " i ", " v ", " vi ", " iv "]
for word in words_to_replace:
    pdf_text = pdf_text.replace(word, "")
pdf_text = pdf_text.replace("  ", " ")
pdf_text

' 14 de noviembre de 2019 Anuncio de Política Monetaria Comunicado de Prensa La Junta de Gobierno del Banco de México ha decidido disminuir en 25 puntos base el objetivo para la Tasa de Interés Interbancaria a un día a un nivel de 750% Durante el tercer trimestre del año continuó la desaceleración de la economía mundial y sus perspectivas de crecimiento han seguido revisándose a la baja Ello en un entorno en el que las inflaciones general y subyacente en las principales economías avanzadas permanecen en niveles inferiores a las metas de sus respectivos bancos centrales En este contexto algunas de estas instituciones han tomado acciones de relajamiento monetario en los últimos meses En su reunión de octubre la Reserva Federal redujo en 25 puntos base el rango objetivo para la tasa de fondos federales y en su comunicación destacó que el rango alcanzado es apropiado considerando los riesgos que enfrenta Por su parte los mercados financieros internacionales presentaron una menor volatilida

In [243]:
set_of_words_in_file = set(pdf_text.split(' '))
list_of_words_in_file = list(pdf_text.split(' '))

In [244]:
clean_list_of_words_in_file = [ word for word in list_of_words_in_file if word != "a" and word != "ante" and word != "en" and word != "sin" and word != "so" and word != "con" and word != "tras" and word != "mediante" and word != "mediante" and word != "versus" and word != "de" and word != "para" and word != "vía" and word != "desde" and word != "por" and word != "ha" and word != "la"]


In [245]:
words_df = pd.DataFrame(clean_list_of_words_in_file)
words_df.columns = ["words"]
words_df['words'].value_counts()

que           38
y             37
el            34
las           29
los           24
              ..
302%           1
ganancias      1
monetario      1
observando     1
Anuncio        1
Name: words, Length: 447, dtype: int64

In [246]:
count = 0
for word in words_to_seek:
    for index in range(len(list_of_words_in_file)):
        if word == list_of_words_in_file[index]:
            print(f"{list_of_words_in_file[index-3]} {list_of_words_in_file[index-2]} {list_of_words_in_file[index-1]} {word} {list_of_words_in_file[index+1]} {list_of_words_in_file[index+2]} {list_of_words_in_file[index+3]}")
    else:
        print(f"{word} isn't in dataframe")

1  La inflación general en octubre
las expectativas de inflación provenientes de encuestas
esperada de la inflación general sugiere que
baja en la inflación no subyacente Por
su parte la inflación subyacente ha continuado
riesgos para la inflación al alza se
mostrada por la inflación subyacente;  aumentos
efecto en la inflación subyacente sea mayor
desviar a la inflación respecto de la
evolución de la inflación respecto a lo
determinantes de la inflación así como sus
que registra la inflación general así como
Para consolidar una inflación baja y estable
comportamiento de la inflación y sus perspectivas
sostenida de la inflación general a la
riesgo para la inflación y sus expectativas
las expectativas de inflación de mediano y
inflación isn't in dataframe


In [247]:
months = {
    "enero": 1,
    "febrero": 2,
    "marzo": 3,
    "abril": 4,
    "mayo": 5,
    "junio": 6,
    "julio": 7,
    "agosto": 8,
    "septiembre": 9,
    "octubre": 10,
    "noviembre": 11,
    "diciembre": 12
}
year = clean_list_of_words_in_file[3]
month = months[clean_list_of_words_in_file[2]]
day = clean_list_of_words_in_file[1]
api_date = f"{year}-{month}-{day}"
api_date

'2019-11-14'

“desaceleración economía mundial” +/-4
“perspectivas crecimiento” +/-4
“inflaciones general subyacente principales economías avanzadas” +5
“inflación principales economías avanzadas” +5
“inflación subyacente principales economías avanzadas” +5
“relajamiento monetario” +/-4
“reserva federal” +5
“comunicación destacó” +6
“mercados financieros internacionales” +6
“volatilidad” +/-4
“posturas acomodaticias principales bancos centrales” +/-4
“riesgos economía global” +/-3
“balance riesgos actividad económica mundial” +/-3
“mercados financieros economías emergentes” +/-4
“cotización peso” +/-4
“tasas interés valores gubernamentales” +6
“política monetaria prudente firme” +/-2
“consoliden sosteniblemente las finanzas públicas” +/-3
“perspectivas crediticias soberana pemex” +/-3
“actividad económica” +/-4
“holgura economía” +/-4
“balance riesgos crecimiento” +5
“inflación general” +/-5
“inflación subyacente” +/-5
“subyacente” +/-5
“persistencia mostrada inflación subyacente”
“aumentos salariales” +/-4
“finanzas públicas” +/-4
“cotización moneda” +/-4
“mantiene incertidumbre cuanto riesgos pudieran desviar inflación respecto trayectoria esperada”
“balance de riesgos para la inflación” +/-3
“incertidumbre” +/-4
“condiciones holgura economía” +/-4
“decidió mayoría” + 4
 “decidió unanimidad” +4
“decidió manera unánime” +5
“un miembro votó” +5
“dos miembros votaron” +5
“tomará las acciones” +8
“la convergencia ordenada y sostenida de la inflación general” +/-6
“postura monetaria prudente” +/-4
“ajustará de manera oportuna y firme” +/-8
“fortalecer el anclaje” +/-5
“presiones costos economía” +/-4

In [248]:
clean_list_of_words_in_file

['',
 '14',
 'noviembre',
 '2019',
 'Anuncio',
 'Política',
 'Monetaria',
 'Comunicado',
 'Prensa',
 'La',
 'Junta',
 'Gobierno',
 'del',
 'Banco',
 'México',
 'decidido',
 'disminuir',
 '25',
 'puntos',
 'base',
 'el',
 'objetivo',
 'Tasa',
 'Interés',
 'Interbancaria',
 'un',
 'día',
 'un',
 'nivel',
 '750%',
 'Durante',
 'el',
 'tercer',
 'trimestre',
 'del',
 'año',
 'continuó',
 'desaceleración',
 'economía',
 'mundial',
 'y',
 'sus',
 'perspectivas',
 'crecimiento',
 'han',
 'seguido',
 'revisándose',
 'baja',
 'Ello',
 'un',
 'entorno',
 'el',
 'que',
 'las',
 'inflaciones',
 'general',
 'y',
 'subyacente',
 'las',
 'principales',
 'economías',
 'avanzadas',
 'permanecen',
 'niveles',
 'inferiores',
 'las',
 'metas',
 'sus',
 'respectivos',
 'bancos',
 'centrales',
 'En',
 'este',
 'contexto',
 'algunas',
 'estas',
 'instituciones',
 'han',
 'tomado',
 'acciones',
 'relajamiento',
 'monetario',
 'los',
 'últimos',
 'meses',
 'En',
 'su',
 'reunión',
 'octubre',
 'Reserva',
 'Fed

In [249]:
count = 0
snippets = dict()
for phrase, value in phrases_to_seek.items():
    if phrase in pdf_text:
        phrase_snippet = phrase.split(" ")[0]
        snippets[f"{phrase_snippet}"] = value
        print(phrase)

comunicación destacó
mercados financieros internacionales
volatilidad
consoliden sosteniblemente las finanzas públicas
actividad económica
inflación general
inflación subyacente
subyacente
aumentos salariales
finanzas públicas
incertidumbre
tomará las acciones
convergencia ordenada y sostenida de la inflación general
postura monetaria prudente
ajustará de manera oportuna y firme
fortalecer el anclaje


In [250]:
snippets

{'comunicación': 6,
 'mercados': 6,
 'volatilidad': 4,
 'consoliden': 3,
 'actividad': 4,
 'inflación': 5,
 'subyacente': 5,
 'aumentos': 4,
 'finanzas': 4,
 'incertidumbre': 4,
 'tomará': 8,
 'convergencia': 6,
 'postura': 4,
 'ajustará': 8,
 'fortalecer': 5}

In [251]:
required_phrases = list()

for snippet, value in snippets.items():
    for word in clean_list_of_words_in_file:
        required_phrase_first = str()
        required_phrase_last = str()
        required_word = str()
        count += 1
        if snippet == word:
            for number in reversed(range(value)):
                required_phrase_first += " " + clean_list_of_words_in_file[count-number]
                required_phrase_last += " " + clean_list_of_words_in_file[count-number+value]
                print(count-number, end=" ")
                print(count-number+value, end=" ")
                print()
            required_phrase = required_phrase_first + required_phrase_last
            
            required_phrases.append(required_phrase)
    count = 0

99 105 
100 106 
101 107 
102 108 
103 109 
104 110 
116 122 
117 123 
118 124 
119 125 
120 126 
121 127 
163 169 
164 170 
165 171 
166 172 
167 173 
168 174 
212 218 
213 219 
214 220 
215 221 
216 222 
217 223 
669 675 
670 676 
671 677 
672 678 
673 679 
674 680 
124 128 
125 129 
126 130 
127 131 
223 227 
224 228 
225 229 
226 230 
341 344 
342 345 
343 346 
197 201 
198 202 
199 203 
200 204 
396 400 
397 401 
398 402 
399 403 
457 462 
458 463 
459 464 
460 465 
461 466 
497 502 
498 503 
499 504 
500 505 
501 506 
531 536 
532 537 
533 538 
534 539 
535 540 
559 564 
560 565 
561 566 
562 567 
563 568 
565 570 
566 571 
567 572 
568 573 
569 574 
575 580 
576 581 
577 582 
578 583 
579 584 
585 590 
586 591 
587 592 
588 593 
589 594 
691 696 
692 697 
693 698 
694 699 
695 700 
723 728 
724 729 
725 730 
726 731 
727 732 
739 744 
740 745 
741 746 
742 747 
743 748 
762 767 
763 768 
764 769 
765 770 
766 771 
805 810 
806 811 
807 812 
808 813 
809 814 
859 864 
860 865 
86

In [252]:
required_phrases

[' fondos federales y su comunicación destacó que el rango alcanzado es apropiado',
 ' Por su parte los mercados financieros internacionales presentaron una menor volatilidad debido',
 ' un mejor desempeño los mercados financieros internacionales las últimas semanas No obstante',
 ' un mejor desempeño los mercados financieros economías emergentes En México si bien',
 ' mayor apetito riesgo los mercados financieros internacionales o caso ratificarse el acuerdo',
 ' una menor volatilidad debido principalmente que se han',
 ' bien episodios volatilidad última decisión política monetaria cotización',
 ' se consoliden sosteniblemente las finanzas públicas',
 ' balance riesgos actividad económica mundial continúa sesgado baja',
 ' el estancamiento actividad económica que se venido observando',
 ' 1  La inflación general octubre fue 302% Destaca que',
 ' cuanto las expectativas inflación provenientes encuestas las corto plazo han',
 ' inciden trayectoria esperada inflación general sugiere que